In [32]:
import geopandas as gpd
import pandas as pd
from scipy.stats import norm 
import numpy as np
from shapely.geometry import Polygon
import string

In [2]:
zones = gpd.read_file("../data/processed_data/zones_delineation/PC_Leiden.gpkg")

In [3]:
zones.area.median()

2043.7616624030316

In [49]:
households_data = pd.read_csv('../data/toy_example/toy_example.csv', names= list(range(0,27)))

In [50]:
households_data = households_data.applymap(lambda x: x + norm.rvs(loc = 0, scale = 1))
households_data = households_data.applymap(lambda x: 1 if x>0.5 else 0)
#toy_example = toy_example.applymap(lambda x: 0 if x<0 else x)

In [55]:
# Making an alphabetical index for the city.
list_char=list(string.ascii_uppercase)
alphabet=list_char+[x+y for x in list_char for y in list_char]
name_columns_1 = alphabet[0:13]
# name_columns_2 = [sub + '_2' for sub in alphabet[0:13]]

In [143]:
city_1 = pd.DataFrame(data = [[np.mean(households_data.loc[2*i:2*i+1,2*j:2*j+1].mean().mean()) for i in range(13)] for j in range(13)],
                      columns = name_columns_1,
                      index = name_columns_1)
                      
city_2 = pd.DataFrame(data = [[np.mean(households_data.loc[2*i+1:2*i+2,2*j+1:2*j+2].mean().mean()) for i in range(13)] for j in range(13)],
                      columns = name_columns_1,
                      index = name_columns_1)
                      #columns = name_columns_2,
                      #index = name_columns_2)

In [ ]:
def compute_res_mix(city_mix):
    res_mix = city_mix.T.stack().reset_index()
    city_mix = city_mix.rename(columns = {0:'res_mix'})
    city_mix['polygon_id'] = city_mix['level_0'] + city_mix['level_1']
    city_mix = city_mix.drop(columns = ['level_0','level_1'])
    return res_mix

In [162]:
res_mix_1 = city_1.T.stack().reset_index()
res_mix_1 = res_mix_1.rename(columns = {0:'res_mix'})
res_mix_1['polygon_id'] = res_mix_1['level_0'] + res_mix_1['level_1']
res_mix_1 = res_mix_1.drop(columns = ['level_0','level_1'])

In [163]:
res_mix_2 = city_2.T.stack().reset_index()
res_mix_2 = res_mix_2.rename(columns = {0:'res_mix'})
res_mix_2['polygon_id'] = res_mix_2['level_0'] + res_mix_2['level_1']
res_mix_2 = res_mix_2.drop(columns = ['level_0','level_1'])

In [242]:
# Generation of a grid.
polygons = []
polygon_id = []
for x in range(13):
    for y in range(13):
        polygons.append(Polygon([(3*x,3*y), (3*(x+1), 3*y), (3*(x+1), 3*(y+1)), (3*x, 3*(y+1))]))
        polygon_id.append(name_columns_1[x] + name_columns_1[y])

grid = gpd.GeoDataFrame({'polygon_id':polygon_id,'geometry':polygons})
grid['geometry'] = grid.centroid

In [243]:
# Computing the shortest paths from cell to cell.
shortest_paths = pd.DataFrame({'from_polygon':[],'to_polygon':[],'distance':[]})

for i in res_mix_1['polygon_id']:
    shortest_paths_i = pd.DataFrame({'from_polygon':[i]*len(res_mix_1),
                                     'to_polygon':res_mix_1['polygon_id'],
                                     'distance':grid.distance(grid.loc[res_mix_1['polygon_id'] == i,'geometry'].values[0])})
    shortest_paths = pd.concat([shortest_paths,shortest_paths_i], ignore_index=True)

In [244]:
shortest_paths['weight'] = 1
shortest_paths['weight'] = shortest_paths['weight'].mask(shortest_paths['distance'] > 0,
                                                         1/shortest_paths['distance']**2)

In [245]:
exposure_1 = res_mix_1.merge(shortest_paths[['from_polygon','to_polygon','weight']], 
                            left_on = 'polygon_id',
                            right_on = 'from_polygon')
exposure_1 = exposure_1.drop(columns = ['from_polygon','polygon_id'])
exposure_1 = exposure_1.rename(columns = {'to_polygon':'polygon_id'})

In [246]:
exposure_1['exposure'] = exposure_1['res_mix']*exposure_1['weight']
exposure_1 = exposure_1[['polygon_id','weight','exposure']].groupby(by = 'polygon_id').sum().reset_index()
exposure_1['exposure'] = exposure_1['exposure'] / exposure_1['weight']

exposure_1 = exposure_1.merge(res_mix_1, on ='polygon_id')

In [241]:
exposure_1.loc[exposure_1['res_mix'] != exposure_1['exposure']]

,polygon_id,weight,exposure,res_mix
0,AA,2.588035,0.555559,0.50
1,AB,3.069598,0.604354,0.75
2,AC,3.257042,0.534050,0.50
3,AD,3.358556,0.582557,0.75
4,AE,3.417348,0.469155,0.50
...,...,...,...,...
164,MI,3.417348,0.487160,0.75
165,MJ,3.358556,0.397585,0.25
166,MK,3.257042,0.448354,0.50
167,ML,3.069598,0.430495,0.50
